In [34]:
from opensea import assets,bundles,common
import json

# JSON keys in API CALL

In [39]:
asset = assets.get_assets(limit=1,verified_only=False)
print("First index asset keys")
asset[0].get_json().keys()

First index asset keys


dict_keys(['id', 'token_id', 'num_sales', 'background_color', 'image_url', 'image_preview_url', 'image_thumbnail_url', 'image_original_url', 'animation_url', 'animation_original_url', 'name', 'description', 'external_link', 'asset_contract', 'permalink', 'collection', 'decimals', 'token_metadata', 'owner', 'sell_orders', 'creator', 'traits', 'last_sale', 'top_bid', 'listing_date', 'is_presale', 'transfer_fee_payment_token', 'transfer_fee', 'related_assets', 'orders', 'auctions', 'supports_wyvern', 'top_ownerships', 'ownership', 'highest_buyer_commitment'])

In [33]:
# In depth look at json, keys such as collection has more subkeys
json_asset = json.dumps(asset_dictionary, indent = 4, sort_keys=True) 
print(json_asset)

{
    "animation_original_url": null,
    "animation_url": null,
    "asset_contract": {
        "address": "0x06012c8cf97bead5deae237070f9587f8e7a266d",
        "asset_contract_type": "non-fungible",
        "buyer_fee_basis_points": 0,
        "created_date": "2018-01-23T04:51:38.832339",
        "default_to_fiat": false,
        "description": "CryptoKitties is a game centered around breedable, collectible, and oh-so-adorable creatures we call CryptoKitties! Each cat is one-of-a-kind and 100% owned by you; it cannot be replicated, taken away, or destroyed.",
        "dev_buyer_fee_basis_points": 0,
        "dev_seller_fee_basis_points": 0,
        "external_link": "https://www.cryptokitties.co/",
        "image_url": "https://lh3.googleusercontent.com/C272ZRW1RGGef9vKMePFSCeKc1Lw6U40wl9ofNVxzUxFdj84hH9xJRQNf-7wgs7W8qw8RWe-1ybKp-VKuU5D-tg=s60",
        "name": "CryptoKitties",
        "nft_version": "1.0",
        "only_proxied_transfers": false,
        "opensea_buyer_fee_basis_poin

If you take a look at opensea > models > asset.py 

``` Python
class Asset:
    def __init__(self, json_data):
        
        """
        Put useful information from json_data into their own variables under Asset class.

        :param json_data: json object returned by the opensea-api
        :type json_data: dict
        """
        # ASSET DETAILS
        self.name = json_data["name"]
        self.description = json_data["description"]
        self.token_id = json_data["token_id"]
        self.asset_url = json_data["permalink"]
        self.image_url = json_data["image_url"]
```

If we want to add say the created_date of the crypto we just need to add 
``` Python
self.created_date = json_data["created_date"]
```
and we will be able to call
``` Python
print(asset.created_date)
```

# A look into the Images from OpenSea

In [2]:
# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)

asset_list = assets.get_assets(limit=50,verified_only=False)

for i in range(len(asset_list)):
    asset = asset_list[0] # Get the first asset obejct from the list

    print(asset.owner.address)
    print(asset.owner.profile_img_url)
    print(asset.owner.username)
    print(asset.owner.config)

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    print(asset.image_url)
    print(asset.get_floor_price()) # Floor price of the collection

0xc8db25e7d49ae0c4f4a63b98203fce908b052cf4
https://storage.googleapis.com/opensea-static/opensea-profile/2.png
Offerzaccept

Rare Whixtensions
Oh hi! I'm Rare Whixtensions. I want to live in a world where people believe that one day cats will rule this planet. I would give it all up for a Chanel purse. Let's pawty!
https://opensea.io/assets/0x06012c8cf97bead5deae237070f9587f8e7a266d/126980
https://lh3.googleusercontent.com/wzrRAgkq0gwq1_jg_RN3_hW1eA8CwPefobfyjQP5QCPxBVOoZX5BY4suaP1htwvVlGE5rR4-z-duWJfQJ8ye-g
0.0
